https://neo4j.com/labs/genai-ecosystem/graphrag-python/

https://neo4j.com/docs/neo4j-graphrag-python/current/user_guide_rag.html#

In [13]:
import sys
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "anatomy-exodus-pigment-colombo-octopus-1216"

In [26]:
from neo4j_graphrag.generation import RagTemplate

prompt_template = RagTemplate(
    system_instructions="""당신은 전문 지식과 폭넓은 정보를 갖춘 최첨단 에이전트입니다. 사용자가 질문을 하면 다음 지침에 따라 응답하세요.
1. **풍부한 정보 제공:**
   - 질문에 대해 가능한 모든 관련 배경 지식과 맥락을 설명하고, 필요한 경우 구체적인 예시와 참고 사항을 포함하세요.
   - 복잡한 주제는 단계별로 세분화하여 설명하며, 각 단계의 중요 포인트를 명확하게 전달하세요.

2. **체계적이고 명확한 응답:**
   - 응답은 명확한 구조(예: 개요, 세부 사항, 결론)와 논리적 순서를 갖추어 작성하세요.
   - 정보의 출처나 근거가 있을 경우 이를 함께 명시하고, 최신 정보에 기반한 답변을 제공하도록 노력하세요.

3. **사용자 맞춤형 설명:**
   - 질문의 맥락과 사용자의 수준에 맞춰 설명의 깊이와 세부 사항을 조절하세요.
   - 추가 설명이나 예시가 필요한 경우, 부가 설명을 제공하여 이해를 돕도록 하세요.

4. **중립적이고 객관적인 자세:**
   - 모든 정보는 사실에 기반하여 객관적으로 전달하며, 다양한 관점이나 의견이 존재하는 경우 이를 균형 있게 소개하세요.
   - 최신 업데이트나 변화가 있는 경우, 그에 따른 주의사항을 명확히 설명하세요.

5. **대화형 인터랙션 유지:**
   - 사용자의 추가 질문이나 요청에 대해 신속하고 유연하게 대응하며, 필요한 경우 추가적인 정보를 요청하여 보다 정확한 답변을 제공할 수 있도록 하세요.

이러한 지침을 준수하여 사용자가 필요로 하는 풍부하고 심도 있는 정보를 제공하는 것이 목표입니다. 항상 사용자에게 최고의 도움이 될 수 있도록 노력하세요.

Domain url is 'https://www.lguplus.com/'.
    """
)

In [24]:
# Neo4j Driver
import neo4j

neo4j_driver = neo4j.GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# LLM and Embedding Model
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

llm = OpenAILLM(
    model_name="gpt-4o",
    model_params={
        "temperature": 0  # turning temperature down for more deterministic results
    }
)

embedder = OpenAIEmbeddings()

In [14]:
# Vector Retriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder
)

vector_rag = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=prompt_template)

temp = vector_rag.search("넷플릭스 요금제에 대해 알려줘", retriever_config={'top_k': 5}, return_context=True)
temp

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RagResultModel(answer='죄송하지만, 제공된 문맥에는 넷플릭스 요금제에 대한 정보가 포함되어 있지 않습니다. 대신, 다른 서비스 요금제에 대한 정보가 포함되어 있습니다. 예를 들어, "V컬러링 바이브 플러스"는 월 8,800원에 제공되며, "티빙 베이직"은 월 9,500원에 제공됩니다. 넷플릭스 요금제에 대한 정보를 원하시면, 넷플릭스 공식 웹사이트를 참조하시기 바랍니다.', retriever_result=RetrieverResult(items=[RetrieverResultItem(content='{\'id\': \'65c346e26a0d167f88329fc4e8fea89c\', \'text\': \'{\\n  "요금내용": "월 8,800원",\\n  "서비스명": "V컬러링 바이브 플러스",\\n  "카테고리코드": "M10053",\\n  "사용여부": "Y",\\n  "ALIAS": "per_mo_service",\\n  "상세주소": "/mobile/plan/addon/addon-media/LRZ0003130",\\n  "서비스설명": "V컬러링과 NAVER VIBE 앱 무제한 듣기를  할인된 금액으로 이용 할 수 있는 서비스",\\n  "서비스코드": "LRZ0003130",\\n  "카테고리명": "영상/음악",\\n  "부가가치세": "부가세 포함 금액",\\n  "구분값": "LRZ0003130",\\n  "IOS이미지": "",\\n  "text": "카테고리명:영상/음악 부가가치세:부가세 포함 금액 서비스코드:LRZ0003130 카테고리코드:M10053 서비스명:V컬러링 바이브 플러스 ALIAS:per_mo_service 요금내용:월 8,800원 서비스설명:V컬러링과 NAVER VIBE 앱 무제한 듣기를  할인된 금액으로 이용 할 수 있는 서비스 IOS이미지: 상세주소:/mobile/plan/addon/addon-media/LRZ0003130 구분값:LRZ0003130 사용여부:Y 안드로이드이미지:",\\n  "안드로이드이미

In [27]:
from neo4j_graphrag.generation import GraphRAG, RagTemplate
from neo4j_graphrag.retrievers import VectorCypherRetriever

graph_retriever = VectorCypherRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
MATCH (node)-[relList]-{1, 2}(nb: !Document)
RETURN nb.id, relList[0]
"""
)

graph_rag = GraphRAG(llm=llm, retriever=graph_retriever, prompt_template=prompt_template)

result = graph_rag.search("넷플릭스 요금제에 대해 알려줘", retriever_config={'top_k': 1}, return_context=True)
result

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RagResultModel(answer='넷플릭스는 다양한 요금제를 제공하여 사용자들이 자신의 필요에 맞게 선택할 수 있도록 하고 있습니다. 넷플릭스 요금제는 주로 다음과 같은 요소에 따라 구분됩니다:\n\n1. **화질 및 동시 시청 가능 인원:**\n   - **베이직 요금제:** 이 요금제는 가장 저렴하며, 한 번에 한 기기에서만 시청할 수 있습니다. 화질은 SD(표준 화질)로 제공됩니다.\n   - **스탠다드 요금제:** 이 요금제는 두 기기에서 동시에 시청할 수 있으며, HD(고화질)로 제공됩니다.\n   - **프리미엄 요금제:** 이 요금제는 네 기기에서 동시에 시청할 수 있으며, UHD(초고화질)로 제공됩니다.\n\n2. **가격:**\n   - 요금은 지역에 따라 다를 수 있으며, 각 요금제의 가격은 넷플릭스의 공식 웹사이트나 앱을 통해 확인할 수 있습니다.\n\n3. **기타 혜택:**\n   - 넷플릭스는 다양한 콘텐츠를 제공하며, 영화, 드라마, 다큐멘터리 등 다양한 장르의 콘텐츠를 무제한으로 스트리밍할 수 있습니다.\n\n넷플릭스 요금제는 사용자의 시청 습관과 필요에 따라 선택할 수 있으며, 언제든지 요금제를 변경하거나 해지할 수 있는 유연성을 제공합니다. 최신 요금제 정보와 가격은 넷플릭스 공식 웹사이트를 통해 확인하는 것이 가장 정확합니다.', retriever_result=RetrieverResult(items=[RetrieverResultItem(content="<Record nb.id='Y' relList[0]=<Relationship element_id='5:e91334ed-cf36-4fa8-a84c-5afca4e086d3:1161928703861589653' nodes=(<Node element_id='4:e91334ed-cf36-4fa8-a84c-5afca4e086d3:1685' labels=frozenset() properties={}>, <Node element_id='4:e91334ed-cf36-4fa8-a84c-5

In [4]:
from langchain_neo4j import Neo4jGraph

enhanced_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    enhanced_schema=True
)

In [10]:
# Text2Cypher Retriever
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.generation import GraphRAG, RagTemplate

text2cypher_retriever = Text2CypherRetriever(
    driver=neo4j_driver,
    llm=llm,
    neo4j_schema=enhanced_graph.schema
)

text2cypher_rag = GraphRAG(
    llm=llm,
    retriever=text2cypher_retriever,
    prompt_template=prompt_template
)

result = text2cypher_rag.search(
    "",
    return_context=True
)
result

RagResultModel(answer="죄송하지만, 'https://www.lguplus.com/' 도메인에서 넷플릭스 요금제에 대한 구체적인 정보를 제공할 수 없습니다. 해당 웹사이트를 직접 방문하여 최신 요금제 정보를 확인하시기 바랍니다.", retriever_result=RetrieverResult(items=[], metadata={'cypher': "cypher\nMATCH (p:Product)-[:HAS_DESCRIPTION]->(d:Description)\nWHERE d.id CONTAINS '넷플릭스'\nRETURN p\n", '__retriever': 'Text2CypherRetriever'}))